In [1]:
# Preprocessing Libraries
import os
import json
import pandas as pd

# Plotting Libraries
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.express as px

In [2]:
def put_evals_to_dfs(folder_whole_path, folder_class_blue_path, folder_class_org_path):

    # The containers of data
    DATA_PATHes = [folder_whole_path, folder_class_blue_path, folder_class_org_path]
    dfs = [{}]*3
    
    # For each container from the containers (which are folder whole or a specific class)
    for idx in range(3):
        DATA_PATH = DATA_PATHes[idx]
        
        # For each file at this container
        for filename in os.listdir(DATA_PATH):
            param_name = filename[5:-5]
            path = DATA_PATH + '/' + filename
            metrics_values = {}
            with open(path, 'r') as f:
                metrics_values = json.load(f)
                metrics_values.pop('bbox_mAP_copypaste')
            
            # Add all the metrics for this filename
            dfs[idx][param_name] = metrics_values
        dfs[idx] = pd.DataFrame(dfs[idx]).T.reset_index()
        
    return dfs

In [3]:
# Get the three dataframes
dfs = put_evals_to_dfs("/home/yanuar/Desktop/Json/Whole/Parameter/eval", "/home/yanuar/Desktop/Json/Blue/Parameter/eval",
                       "/home/yanuar/Desktop/Json/Org/Parameter/eval")

In [4]:
# Order the parameters
def evaluate(s):
    lista = s.split('_')
    value = 0
    if lista[0] == 'Su':
        value += 200
    else:
        value += 100
        
    mapping = {str(num)+'m':v for num, v in zip(range(4, 11, 2), range(10, 41, 10))}
    value += mapping[lista[1]]
    
    mapping = {letters:v for letters, v in zip(['Co', 'Gr', 'SG', 'So'], range(1, 5))}
    value += mapping[lista[2]]
    return value


In [5]:
val_name = [str(evaluate(s))+s for s in dfs[0]['index']]
val_name = sorted(val_name, key=lambda x : x[:3])
val_name = list(map(lambda x : x[3:], val_name))

In [6]:
# Order parameters on the dataframes based on the above order
for idx in range(3):
    dfs[idx]["index"] = pd.Categorical(dfs[idx]["index"], categories = val_name)
    dfs[idx] = dfs[idx].sort_values(by = "index")

In [7]:
# Metrics
metrics = ['bbox_mAP', 'bbox_mAP50', 'bbox_mAP75', 'bbox_mAP_s', 'bbox_mAP_m', 'bbox_mAP_l', 'bbox_mAP50_s', 'bbox_mAP50_m', 'bbox_mAP50_l']
wanted_metrics = ['bbox_mAP', 'bbox_mAP50']

In [10]:
# Prepare the main Dataframe which will contain raw for each parameter (total 32) and its columns will be 
# bbox_mAP for each class and bbox_mAP50 for each class (total 3)
classes_names = ['Whole', 'Blue', 'Orignial']
main_df = pd.DataFrame()
main_df['parameter'] = dfs[0]['index']
for metric in wanted_metrics:
    for idx in range(3):
        main_df[metric[5:]+'_Class_'+classes_names[idx]] = dfs[idx][metric]


colorscale=None
colorscale="rdylgn"

fig = go.Figure(data=go.Heatmap(
                   z=main_df.iloc[:, 1:],
                   x=main_df.columns[1:],
                   y=main_df['parameter'], text=main_df.iloc[:, 1:], texttemplate="%{text}",
                   hoverongaps = False, colorscale= colorscale),
                layout=go.Layout(height=1600, width=800, yaxis={'tickfont':{'size':23, 'family': 'Old Standard TT, serif'}},
                                 xaxis={'tickfont':{'size':20, 'family': 'Old Standard TT, serif'}}))

fig.update_layout(title='''<b>               mAP             mAP50<b>''', titlefont={'size': 35}, template='plotly_white', yaxis={'title':'<b>Parameter<b>', 'titlefont':{'size':30}})
annotations = []
annotations.append(dict(
                        x=1, y=32,
                        text=' Whole Class     Blue Class    Original Class',
                        font=dict(family='Arial', size=12, color='black'),
                        showarrow=False))

annotations.append(dict(
                        x=4, y=32,
                        text=' Whole Class     Blue Class    Original Class',
                        font=dict(family='Arial', size=12, color='black'),
                        showarrow=False))


fig.update_layout(annotations=annotations)
fig.show()

In [9]:
main_df

,parameter,mAP_Class_Whole,mAP_Class_Blue,mAP_Class_Orignial,mAP50_Class_Whole,mAP50_Class_Blue,mAP50_Class_Orignial
28,Ov_4m_Co,0.757,0.751,0.763,1.000,1.000,1.000
7,Ov_4m_Gr,0.235,0.326,0.144,0.475,0.736,0.215
14,Ov_4m_SG,0.725,0.703,0.748,0.993,1.000,0.986
25,Ov_4m_So,0.788,0.773,0.802,0.997,1.000,0.995
2,Ov_6m_Co,0.724,0.719,0.730,1.000,1.000,1.000
4,Ov_6m_Gr,0.156,0.265,0.048,0.376,0.639,0.113
3,Ov_6m_SG,0.667,0.655,0.680,0.973,0.994,0.953
24,Ov_6m_So,0.718,0.711,0.725,0.991,1.000,0.983
5,Ov_8m_Co,0.706,0.709,0.703,0.996,1.000,0.991
30,Ov_8m_Gr,0.136,0.190,0.082,0.347,0.524,0.170
